In [1]:
%run LongTrail_BuildGraph.ipynb

C:\ProgramData\Anaconda3\lib\site-packages\networkx\drawing\nx_pylab.py:579: MatplotlibDeprecationWarning: 
The iterable function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use np.iterable instead.
  if not cb.iterable(width):


<Figure size 10000x10000 with 1 Axes>

In [2]:
from gurobipy import *

# Create a new model
m = Model()

Academic license - for non-commercial use only


In [3]:
#for each edge, create a variable in the model
gurobiVars = dict()
obj = LinExpr()

for u,v,a in G.edges(data=True):
    thisEdgeName = str(u) + "_" + str(v) if u < v else str(v) + "_" + str(u)
    thisVar = m.addVar(vtype=GRB.INTEGER, name=thisEdgeName)
    gurobiVars[thisEdgeName] = thisVar
    obj += (a['weight'] * (thisVar+1))
    
m.setObjective(obj, GRB.MINIMIZE)

In [4]:
#now add constraints. A eulerian tour requires (and will always exist if) no more than two nodes are odd 
#(because the two odd nodes are the start and end node)
#Let's assume that the two odd nodes here are the southern terminus and northern terminus of the trail

#to do this, iterate over all nodes. If the node is currently odd, state that the number of edges added that
#are adjacent to that node should be an odd number. If it's even, the number of edges added should be even.

#note that Gurobi doesn't allow modulo operator, so to solve that, declare a dedicated int variable for each
#constraint, and then express each constraint in the form X + Y = 2z (for evenness)
#                                                      or X + Y = 2z+1 (for oddness)
for n in G.nodes():
    constraintForThisNode = LinExpr()
    thisNodeEvenOddVar = m.addVar(vtype=GRB.INTEGER, name="EvennessForNode" + str(n))
    for u,v,a in G.edges(n, data=True):
        edgeName = str(u) + "_" + str(v) if u < v else str(v) + "_" + str(u)
        thisEdge = gurobiVars[edgeName]
        constraintForThisNode += thisEdge
    if(n == 0):#node 0 is odd and should stay odd (it's the starting point)
        m.addConstr(constraintForThisNode == (2*thisNodeEvenOddVar))
    elif(n==211):#node 211 is even and we want to make it odd (it's the ending point)
        m.addConstr(constraintForThisNode == ((2*thisNodeEvenOddVar)+1))
    elif(G.degree(n) % 2 == 1): #all other nodes that are odd should become even
        m.addConstr(constraintForThisNode == ((2*thisNodeEvenOddVar)+1))
    else:#all nodes that are currently even should stay even
        m.addConstr(constraintForThisNode == (2*thisNodeEvenOddVar))        

In [5]:
#do the thing
m.optimize()

Optimize a model with 201 rows, 428 columns and 655 nonzeros
Variable types: 0 continuous, 428 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e-01, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 74 rows and 148 columns
Presolve time: 0.00s
Presolved: 127 rows, 280 columns, 433 nonzeros
Variable types: 0 continuous, 280 integer (0 binary)

Root relaxation: objective 5.633000e+02, 54 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  563.30000    0   12          -  563.30000      -     -    0s
     0     0  570.20000    0    6          -  570.20000      -     -    0s
     0     0  571.20000    0    2          -  571.20000      -     -    0s
H    0     0                     573.0000000  571.20000  0.31%     -    0s
H    0     0                     572.80000

In [6]:
#print the results, sorted by variable name
vList = m.getVars()
vList.sort(key=lambda var: int(var.varName[:var.varName.find("_",0)]) if var.varName.find("_",0) > 0 else 1000 )


for v in vList:
    print('%s %g' % (v.varName, v.x))

print('Obj: %g' % m.objVal)

0_2 0
1_2 1
2_3 0
3_4 0
4_5 0
5_6 0
6_10 0
7_9 1
8_9 1
9_12 1
10_12 0
10_11 1
12_13 0
13_15 0
14_23 1
15_17 0
16_21 1
17_18 0
18_19 1
18_20 0
19_24 1
20_21 0
21_22 0
22_23 0
22_25 1
23_25 0
23_27 1
25_26 0
26_29 0
29_30 0
30_32 0
31_32 1
32_38 0
32_33 1
33_35 0
33_36 0
34_35 1
35_36 0
36_37 1
37_39 1
38_39 0
39_41 0
40_44 1
40_41 1
41_42 0
42_43 0
43_46 0
44_45 1
46_49 0
46_47 1
49_51 1
49_53 0
53_54 0
54_55 0
55_56 0
56_57 0
57_58 0
58_60 0
59_63 1
60_62 0
61_63 1
62_64 0
63_64 1
64_66 0
65_66 1
66_67 0
67_68 1
67_69 0
69_70 0
69_74 1
70_71 1
70_72 0
72_76 0
73_75 1
74_77 1
75_78 -0
75_76 -0
77_78 -0
77_80 0
79_80 1
80_84 0
81_84 1
81_83 1
83_86 1
84_85 0
85_87 0
87_88 1
87_90 0
89_91 1
90_91 0
91_93 0
92_93 1
93_94 0
94_97 0
94_95 1
96_98 1
97_98 1
97_99 0
98_100 1
99_100 0
100_101 0
101_103 0
102_103 1
103_105 0
104_105 1
105_106 0
106_107 0
106_108 1
107_109 1
107_108 0
108_110 0
110_111 0
111_112 1
111_114 0
113_114 1
114_116 0
115_118 1
116_119 -0
116_117 1
117_119 -0
117_118 0
1

In [7]:
#so the answer is that if you do it all in one trip, the total cost of trails that you need to double-back on
#is 135.1. That's in addition to walking each path once, so add the 437.5 = 572.6 total miles